In [33]:
import pandas as pd
import numpy as np
import matplotlib as plt

Import datasets

In [34]:
d13 = pd.read_stata('./data/2013', convert_categoricals=False)
d14 = pd.read_stata('./data/2014', convert_categoricals=False)
d15 = pd.read_stata('./data/2015', convert_categoricals=False)
d16 = pd.read_stata('./data/2016', convert_categoricals=False)
d17 = pd.read_stata('./data/2017', convert_categoricals=False)
d18 = pd.read_stata('./data/2018', convert_categoricals=False)
d19 = pd.read_stata('./data/2019', convert_categoricals=False)
d20 = pd.read_stata('./data/2020', convert_categoricals=False)
d21 = pd.read_stata('./data/2021', convert_categoricals=False)
d22 = pd.read_stata('./data/2022', convert_categoricals=False)

Recode pre-2017 LFSSTAT variables and rename EDUC90 to EDUC

In [84]:
pre = pd.concat([d13, d14, d15, d16])

def recode(LFSSTAT):
	if LFSSTAT == 4:
		return 3
	elif LFSSTAT == 5:
		return 3
	elif LFSSTAT == 6:
		return 4
	else:
		return LFSSTAT

pre.rename(columns={'EDUC90':'EDUC'}, inplace=True)

pre['LFSSTAT'] = pre['LFSSTAT'].apply(recode)

Append the rest of the data to the master

In [85]:
post = pd.concat([d17, d18, d19, d20, d21, d22])
master = pd.concat([pre, post])

Keep only prime age - 20 to 64 & Non-students

In [86]:
master = master[(master['AGE_12'] >= 2) & (master['AGE_12'] <= 10) & (master['SCHOOLN'] == 1)]

    QUESTION 1

        1a) Educational attainment

Generating new 'edu' variable and populating it

In [87]:
master['edu'] = np.nan

In [88]:
def recodeEDU(EDUC):
    if EDUC <= 1:
        return 1
    elif (EDUC == 2 or EDUC == 3):
        return 2
    elif EDUC == 4:
        return 3
    elif EDUC == 5:
        return 4
    elif EDUC == 6:
        return 5
    else:
        return EDUC

master['edu'] = master['EDUC'].apply(recodeEDU)

In [95]:
master['edu'].value_counts(normalize=True).sort_index()

1    0.101760
2    0.269193
3    0.379968
4    0.173800
5    0.075279
Name: edu, dtype: float64

    1b) Educational attainment over time

In [102]:
pd.crosstab(master['SURVYEAR'], master['edu'], normalize='index')

edu,1,2,3,4,5
SURVYEAR,,,,,
2013,0.120462,0.288373,0.372295,0.154795,0.064075
2014,0.118577,0.289459,0.371570,0.155208,0.065185
2015,0.112379,0.277708,0.380284,0.163051,0.066579
2016,0.110637,0.274003,0.379812,0.164896,0.070652
2017,0.103271,0.275455,0.382741,0.168699,0.069833
2018,0.102338,0.269140,0.383913,0.170180,0.074429
2019,0.097701,0.262525,0.383998,0.177943,0.077833
2020,0.087580,0.256509,0.389631,0.183827,0.082453
2021,0.081836,0.256831,0.384399,0.193931,0.083002


dasda

In [104]:
print('hello')

hello
